In [ ]:
import os
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import warnings
from utils import names, name_colors, name_linestyles, name_shapes, name_order

# Constants
FOLDER_PATH = "../results/"
MAX_RECENT_FILES = 10
COLOR_CYCLE = [
    "#377eb8",
    "#ff7f00",
    "#4daf4a",
    "#f781bf",
    "#a65628",
    "#984ea3",
    "#999999",
    "#e41a1c",
    "#dede00",
]

# Matplotlib configuration
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(color=COLOR_CYCLE)

# Suppress FutureWarning for Seaborn
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

# Get the most recent CSV files
csv_files = glob.glob(os.path.join(FOLDER_PATH, "*.csv"))
csv_files.sort(key=os.path.getmtime, reverse=True)
recent_files = csv_files[:MAX_RECENT_FILES]

print("Most recent CSV files:")
for file in recent_files:
    print(file)

In [ ]:
file_to_read = " " # input file path
try:
    # Load data from the CSV file
    df= pd.read_csv(file_to_read)
    df.drop(
        columns=["iterations", "experiment", "data_folder", "n_samples"],
        inplace=True,
    )
except FileNotFoundError:
    print(f"File {file_to_read} not found.")
    df = None


if df is not None:
    # Update method names using the mapping from the 'names' dictionary
    df["method"] = df["method"].apply(lambda n: names.get(n, n))
    print("Updated method names:", np.unique(df["method"]))


selected_methods = [
    "HGIC  (KCIT)",
    "Ours (Quadratic)",
    "Transp. test (KCIT)",
]

In [ ]:
grouped_df = df.groupby(by=['conf_strength', 'method', 'n_observed_confounders', 'degree']).mean().reset_index()
grouped_df = grouped_df.pivot(index=['method'], columns=['conf_strength', 'n_observed_confounders', 'degree'], values=['detection'])
print(grouped_df)

In [ ]:
# Get the unique methods and filter the dataframe accordingly
df = df[df["method"].isin(selected_methods)]

sns.set_context("talk", font_scale=1.0)  # Adjust font scale if needed

for degree in np.unique(df["degree"]):
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)  # Two subplots, same y-axis

    for i, conf_strength in enumerate([0, 1]):  # Loop over conf_strength 0 and 1
        ax = axes[i]

        # Filter data for the current confounding strength
        tmp_df = df[(df["conf_strength"] == conf_strength) & (df["degree"] == degree)]

        # Drop unwanted columns and compute mean and standard error
        tmp_df = (
            tmp_df.drop(columns=["pval"])
            .groupby(by=["method", "n_observed_confounders"])
            .agg(detection_mean=("detection", "mean"), count=("detection", "count"))
            .reset_index()
        )
        tmp_df["detection_se"] = np.sqrt(
            tmp_df["detection_mean"] * (1 - tmp_df["detection_mean"]) / tmp_df["count"]
        )

        # Line plot with error bars
        sns.lineplot(
            data=tmp_df,
            x="n_observed_confounders",
            y="detection_mean",
            hue="method",
            style="method",
            markers=name_shapes,
            palette=name_colors,
            dashes=name_linestyles,
            hue_order=[mname for mname in name_order if mname in tmp_df.method.unique()],
            ax=ax
        )

        for method, group in tmp_df.groupby("method"):
            ax.errorbar(
                group["n_observed_confounders"],
                group["detection_mean"],
                yerr=group["detection_se"],
                fmt="none",  # No markers on error bars
                capsize=5,
                color=name_colors[method],
            )

        ax.set_ylim([-0.02, 1.02])
        ax.axhline(0.05, linestyle="--", color="black")
        ax.set_xlabel("Number of observed confounders")
        if i == 0:
            ax.set_ylabel("Falsification rate")
        else:
            ax.set_ylabel("")
        ax.legend().remove()
        ax.set_title(f"{"No unmeasured confounder" if conf_strength == 0.0 else 'Five confounders present'}")

    # Common legend below the plots
    handles, labels = axes[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, -0.1), ncol=3, fontsize="small")

    plt.tight_layout()
    plt.savefig(f"figures/expTwins-degree{degree}-new.pdf", bbox_inches="tight", dpi=300)
    plt.show()
